<a href="https://colab.research.google.com/github/vksepm/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/blob/main/code_01_XX%20A%20basic%20Llamaindex%20agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install pre-requisite packages for the course

Please ensure that this cell is run, to install all pre-requisite packages needed for subsequent code examples.

In [1]:
!pip install -qU python-dotenv
!pip install -qU llama-index
!pip install -qU llama-index-llms-openai
!pip install -qU llama-index-embeddings-openai
!pip install -qU llama-index-llms-azure-openai
!pip install -qU llama-index-embeddings-azure-openai
!pip install -qU llama-index-agent-introspective
!pip install -qU llama-index-program-openai
!pip install -qU llama-index-readers-file
!pip install -qU pydantic
!pip install -qU llama-index-core
!pip install -qU llama-index-utils-workflow
!pip install -qU llama-index-tools-wikipedia
!pip install -qU llama-index-readers-json
!pip install -qU llama-index-readers-file
!pip install -qU pymupdf
!pip install -qU llama-index-tools-google
!pip install -qU nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━

### 01.04. Setup LLM and Tools in LlamaIndex

#### Setup the LLM

In [2]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()


import os
from google.colab import userdata

#Setup the environment variables
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')

#Note : Function calling support only available in GPT-4+
#Create the LLM object
llm=AzureOpenAI(
    model="gpt-4o-mini",
    deployment_name="gpt-4o-mini",
    api_version="2024-08-01-preview",
)

#### Set up Functions

In [3]:
#define a couple of functions
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y

### 01.05. Setup and execute an AI Agent

In [4]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.base import ReActAgent

#Create function tools
sum_tool = FunctionTool.from_defaults(find_sum)
product_tool=FunctionTool.from_defaults(find_product)

#Create a pre-built ReAct agent
simple_agent = ReActAgent.from_tools(
                [sum_tool, product_tool],
                llm=llm,
                verbose=True #Set verbose for detailed logs
                )

#### Test the Agent

In [5]:
response = simple_agent.chat("What is 2 + 2? ")

> Running step 1c77c6aa-ad10-4915-9d0a-2c819fa9f1ff. Step input: What is 2 + 2? 
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_sum
Action Input: {'x': 2, 'y': 2}
Observation: 4
> Running step 0e2e2773-b857-4c2d-81a9-5f241c6cb6e7. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 2 + 2 equals 4.


In [6]:
response = simple_agent.chat("Multiple 2 by 3 and then add 4")

> Running step dbba45f4-9864-4073-bfb2-e5019a5babcc. Step input: Multiple 2 by 3 and then add 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_product
Action Input: {'x': 2, 'y': 3}
Observation: 6
> Running step 460ac055-889c-40c4-9588-507f8a0a80b5. Step input: None
Thought: Now that I have the product of 2 and 3, which is 6, I need to add 4 to it.
Action: find_sum
Action Input: {'x': 6, 'y': 4}
Observation: 10
> Running step 1514cedf-e4f8-4a9f-bcb9-4dffb94a3ee8. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of multiplying 2 by 3 and then adding 4 is 10.
